### Global Parameters

In [ ]:
SEED = 41

### Load Data

In [47]:
import os
import numpy as np
import pandas as pd
from PIL import ImageEnhance
from PIL import Image
import random
import matplotlib.pyplot as plt

def random_crop(image, crop_size):
    """
    Randomly crop the image to the specified size.

    Parameters:
        image (PIL.Image): Input image.
        crop_size (int): Size of the cropped image.

    Returns:
        PIL.Image: Cropped image.
    """
    width, height = image.size
    left = random.randint(0, width - crop_size)
    upper = random.randint(0, height - crop_size)
    right = left + crop_size
    lower = upper + crop_size
    return image.crop((left, upper, right, lower))

def random_rotation(image, max_angle=30):
    """
    Randomly rotate the image within a specified angle range.

    Parameters:
        image (PIL.Image): Input image.
        max_angle (int): Maximum rotation angle in degrees.

    Returns:
        PIL.Image: Rotated image.
    """
    angle = random.randint(-max_angle, max_angle)
    return image.rotate(angle)

def color_jitter(image, brightness=0.2, contrast=0.2, color=0.2):
    """
    Apply random color jittering to the image.

    Parameters:
        image (PIL.Image): Input image.
        brightness (float): Brightness adjustment factor range (e.g., 0.2 means [0.8, 1.2]).
        contrast (float): Contrast adjustment factor range.
        color (float): Color adjustment factor range.

    Returns:
        PIL.Image: Color-jittered image.
    """
    # Adjust brightness
    enhancer = ImageEnhance.Brightness(image)
    factor = random.uniform(1 - brightness, 1 + brightness)
    image = enhancer.enhance(factor)

    # Adjust contrast
    enhancer = ImageEnhance.Contrast(image)
    factor = random.uniform(1 - contrast, 1 + contrast)
    image = enhancer.enhance(factor)

    # Adjust color
    enhancer = ImageEnhance.Color(image)
    factor = random.uniform(1 - color, 1 + color)
    image = enhancer.enhance(factor)

    return image

def aug_data(X, y, seed=None):
    """
    Augment the data by applying random cropping, random rotation, horizontal flipping, and color jittering.
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    X_aug = []
    y_aug = []

    for i in range(len(X)):
        # Reshape to original image shape (H, W, C)
        img = X[i].reshape(3, 32, 32).transpose(1, 2, 0)  # (32, 32, 3)
        label = y[i]

        # Convert to PIL Image for easier augmentation
        pil_img = Image.fromarray(img)

        # Append original image
        X_aug.append(np.array(pil_img).flatten())
        y_aug.append(label)

        # Apply augmentations
        # Horizontal flip
        flipped_img = pil_img.transpose(Image.FLIP_LEFT_RIGHT)
        X_aug.append(np.array(flipped_img).flatten())
        y_aug.append(label)

        # # Random crop
        # if random.random() < 0.1:
        #     crop_size = random.randint(24, 32)  # Random crop size between 24x24 and 32x32
        #     cropped_img = random_crop(pil_img, crop_size).resize((32, 32))
        #     X_aug.append(np.array(cropped_img).flatten())
        #     y_aug.append(label)

        # # Random rotation
        # if random.random() < 0.1:
        #     rotated_img = random_rotation(pil_img).resize((32, 32))
        #     X_aug.append(np.array(rotated_img).flatten())
        #     y_aug.append(label)

        # # Color jittering
        # if random.random() < 0.1:
        #     jittered_img = color_jitter(pil_img)
        #     X_aug.append(np.array(jittered_img).flatten())
        #     y_aug.append(label)

    return np.array(X_aug), np.array(y_aug)


csv_path, csv_test_path = "../../data/train.csv", "../../data/test.csv"
img_dir, img_dir_test = "../../data/train_ims", "../../data/test_ims"

data_train = pd.read_csv(csv_path)
print(data_train)
data_test = pd.read_csv(csv_test_path)

X_train, y_train, X_test, y_test = [], [], [], []

for _, row in data_train.iterrows():
    img_path = os.path.join(img_dir, row.iloc[0])
    label = int(row.iloc[1])
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).flatten()
    X_train.append(img)
    y_train.append(label)

for _, row in data_test.iterrows():
    img_path = os.path.join(img_dir_test, row.iloc[0])
    label = int(row.iloc[1])
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).flatten()
    X_test.append(img)
    y_test.append(label)
# Example usage
print("Original data size:", len(X_train), len(y_train))
print("Test data size:", len(X_test), len(y_test))
# Augment the training data
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train, y_train = aug_data(X_train, y_train, SEED)
print(X_train.shape, y_train.shape)

X_test = np.array(X_test)
y_test = np.array(y_test)

           im_name  label
0      000231c.jpg      2
1      0002574.jpg      5
2      00027d5.jpg      7
3      000304e.jpg      3
4      00047fc.jpg      7
...            ...    ...
49995  d507197.jpg      5
49996  d508429.jpg      1
49997  d508cb7.jpg      9
49998  d509167.jpg      3
49999  d509c42.jpg      3

[50000 rows x 2 columns]
Original data size: 50000 50000
Test data size: 10000 10000
(100000, 3072) (100000,)


### Size of dataset

In [48]:
print("The shape of the training set (after flatten 3 * 32 * 32): ", X_train.shape)
print("The shape of the testing set (after flatten 3 * 32 * 32): ", X_test.shape)
print("The size of the label of the training set: ", y_train.shape)
print("The size of the label of the test set: ", y_test.shape)

The shape of the training set (after flatten 3 * 32 * 32):  (100000, 3072)
The shape of the testing set (after flatten 3 * 32 * 32):  (10000, 3072)
The size of the label of the training set:  (100000,)
The size of the label of the test set:  (10000,)


### Calculate the number of the labels of each class

In [49]:
# Count the labels
count_labels = np.unique(y_train, return_counts=True)

# Create DataFrame
label_counts_df = pd.DataFrame({
    "Label": count_labels[0],
    "Count": count_labels[1]
}).set_index("Label")

# Print DataFrame
print("Count of each label:")
print(label_counts_df)

Count of each label:
       Count
Label       
0      10076
1      10032
2      10064
3       9982
4       9964
5       9934
6       9970
7       9996
8      10004
9       9978


### Draw Counts

In [50]:
print("Other Statistics for the training set:")
print(pd.Series(y_train).describe())

Other Statistics for the training set:
count    100000.000000
mean          4.492580
std           2.875376
min           0.000000
25%           2.000000
50%           4.000000
75%           7.000000
max           9.000000
dtype: float64


### Data Augmentation

In [51]:
import numpy as np
from PIL import Image, ImageEnhance
import random

# def random_crop(image, crop_size):
#     """
#     Randomly crop the image to the specified size.

#     Parameters:
#         image (PIL.Image): Input image.
#         crop_size (int): Size of the cropped image.

#     Returns:
#         PIL.Image: Cropped image.
#     """
#     width, height = image.size
#     left = random.randint(0, width - crop_size)
#     upper = random.randint(0, height - crop_size)
#     right = left + crop_size
#     lower = upper + crop_size
#     return image.crop((left, upper, right, lower))

# def random_rotation(image, max_angle=30):
#     """
#     Randomly rotate the image within a specified angle range.

#     Parameters:
#         image (PIL.Image): Input image.
#         max_angle (int): Maximum rotation angle in degrees.

#     Returns:
#         PIL.Image: Rotated image.
#     """
#     angle = random.randint(-max_angle, max_angle)
#     return image.rotate(angle)

# def color_jitter(image, brightness=0.2, contrast=0.2, color=0.2):
#     """
#     Apply random color jittering to the image.

#     Parameters:
#         image (PIL.Image): Input image.
#         brightness (float): Brightness adjustment factor range (e.g., 0.2 means [0.8, 1.2]).
#         contrast (float): Contrast adjustment factor range.
#         color (float): Color adjustment factor range.

#     Returns:
#         PIL.Image: Color-jittered image.
#     """
#     # Adjust brightness
#     enhancer = ImageEnhance.Brightness(image)
#     factor = random.uniform(1 - brightness, 1 + brightness)
#     image = enhancer.enhance(factor)

#     # Adjust contrast
#     enhancer = ImageEnhance.Contrast(image)
#     factor = random.uniform(1 - contrast, 1 + contrast)
#     image = enhancer.enhance(factor)

#     # Adjust color
#     enhancer = ImageEnhance.Color(image)
#     factor = random.uniform(1 - color, 1 + color)
#     image = enhancer.enhance(factor)

#     return image

# def aug_data(X, y, seed=None):
#     """
#     Augment the data by applying random cropping, random rotation, horizontal flipping, and color jittering.
#     """
#     if seed is not None:
#         random.seed(seed)
#         np.random.seed(seed)

#     X_aug = []
#     y_aug = []

#     for i in range(len(X)):
#         # Reshape to original image shape (H, W, C)
#         img = X[i].reshape(3, 32, 32).transpose(1, 2, 0)  # (32, 32, 3)
#         label = y[i]

#         # Convert to PIL Image for easier augmentation
#         pil_img = Image.fromarray(img)

#         # Append original image
#         X_aug.append(np.array(pil_img).flatten())
#         y_aug.append(label)

#         # Apply augmentations
#         # Horizontal flip
#         flipped_img = pil_img.transpose(Image.FLIP_LEFT_RIGHT)
#         X_aug.append(np.array(flipped_img).flatten())
#         y_aug.append(label)

#         # Random crop
#         if random.random() < 0.1:
#             crop_size = random.randint(24, 32)  # Random crop size between 24x24 and 32x32
#             cropped_img = random_crop(pil_img, crop_size).resize((32, 32))
#             X_aug.append(np.array(cropped_img).flatten())
#             y_aug.append(label)

#         # Random rotation
#         if random.random() < 0.1:
#             rotated_img = random_rotation(pil_img).resize((32, 32))
#             X_aug.append(np.array(rotated_img).flatten())
#             y_aug.append(label)

#         # Color jittering
#         if random.random() < 0.1:
#             jittered_img = color_jitter(pil_img)
#             X_aug.append(np.array(jittered_img).flatten())
#             y_aug.append(label)

#     return np.array(X_aug), np.array(y_aug)

# # Example usage
# X_train_aug, y_train_aug = aug_data(X_train, y_train, SEED)

### Data after augmentation

In [52]:
# print("Augmented training set shape: ", X_train_aug.shape)
# print("Augmented training set labels shape: ", y_train_aug.shape)
# # Count the labels
# count_labels = np.unique(y_train_aug, return_counts=True)
# # Create DataFrame
# label_counts_df = pd.DataFrame({
#     "Label": count_labels[0],
#     "Count": count_labels[1]
# }).set_index("Label")
# # Print DataFrame
# print("Count of each label in augmented training set:")
# print(label_counts_df)

### Resize

In [53]:
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from tqdm import tqdm
import cv2

# def resize_images(images, target_size=(128, 128)):
#     """
#     Resize a list or array of images to the target size.

#     Parameters:
#         images (numpy.ndarray): Input images, shape (N, H, W, C) or (N, H*W*C).
#         target_size (tuple): Target size (width, height), default is (128, 128).

#     Returns:
#         numpy.ndarray: Resized images, shape (N, target_size[1], target_size[0], C).
#     """
#     resized_images = []
#     for img in images:
#         # If the image is flattened, reshape it to (H, W, C)
#         if len(img.shape) == 1:
#             img = img.reshape(32, 32, 3)  # Adjust this based on your original image shape

#         # Resize the image
#         resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
#         resized_images.append(resized_img)

#     return np.array(resized_images)

# X_train_aug_resized = resize_images(X_train_aug, target_size=(128, 128))
# print("Resized images shape:", X_train_aug_resized.shape)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


### Split the data

In [54]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_val.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_val.shape)

X_train = X_train.reshape(X_train.shape[0], -1)  # Flatten the images
X_val = X_val.reshape(X_val.shape[0], -1)  # Flatten the images
print("Flattened training set shape:", X_train.shape)
print("Flattened testing set shape:", X_val.shape)

Training set shape: (80000, 3072)
Testing set shape: (20000, 3072)
Training labels shape: (80000,)
Testing labels shape: (20000,)
Flattened training set shape: (80000, 3072)
Flattened testing set shape: (20000, 3072)


## Feature Extraction

### Extract original data

In [55]:
def image_statistics_extractor(image):
    """
    Extract statistical features from the original image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).

    Returns:
        numpy.ndarray: Statistical feature vector for the input image.
    """
    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)  # Adjust this based on your original image shape

    resized_image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_CUBIC)

    return resized_image.flatten()
    

### SIFT

In [56]:
import cv2
import numpy as np

def SIFT_extractor(image, n_features=128):
    """
    Extract SIFT features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).

    Returns:
        numpy.ndarray: SIFT descriptors for the input image. If no descriptors are found, returns an empty array.
    """
    sift = cv2.SIFT_create()

    if not isinstance(image, np.ndarray):
        image = np.array(image)

    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3) 
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    # If no descriptors are found, return an empty array
    if descriptors is None:
        return np.zeros((1, n_features)).flatten()

    return np.mean(descriptors, axis=0)

### HOG

In [57]:
from skimage.feature import hog
import cv2
import numpy as np

def HOG_extractor(image, target_size=(128, 128)):
    """
    Extract HOG features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting HOG features.

    Returns:
        numpy.ndarray: Flattened HOG feature vector for the input image.
    """
    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)  # Adjust this based on your original image shape
    
    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

    # # Convert to grayscale (HOG works on single-channel images)
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)
    # print("Gray image shape:", gray_img.shape)


    # Split the image into three channels (R, G, B)
    channels = cv2.split(resized_img)
    # print("Channels shape:", [channel.shape for channel in channels])

    # Compute HOG features
    hog_kwargs = {
        "_winSize": (128, 128),
        "_blockSize": (64, 64),
        "_blockStride": (16, 16),
        "_cellSize": (16, 16),
        "_nbins": 10,
        "_derivAperture": 1,
        "_winSigma": -1,
        "_histogramNormType": 0,
        "_L2HysThreshold": 0.2,
        "_gammaCorrection": True,
        "_nlevels": 64,
        "_signedGradient": True
        
    }   

    hog = cv2.HOGDescriptor(**hog_kwargs)

    # Compute HOG features
    hog_features = []
    for channel in channels:
        # Compute HOG features for the current channel
        hog_feature = hog.compute(channel)
        # print ("HOG feature shape:", hog_feature.shape)
        hog_features.append(hog_feature.flatten())
    
    # gray_hog_feature = hog.compute(gray_img)
    # print ("Gray HOG feature shape:", gray_hog_feature.shape)

    # Flatten the HOG feature vector
    return np.concatenate(hog_features)

### EOG

In [58]:
def EOG_extractor(image, target_size=(32, 32)):
    """
    Extract Edge Orientation Gradient (EOG) features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting EOG features.

    Returns:
        numpy.ndarray: Flattened EOG feature vector for the input image.
    """
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)
    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_CUBIC)
    # Convert to grayscale
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray_img, threshold1=100, threshold2=200)
    # Compute gradients in x and y directions
    grad_x = cv2.Sobel(edges, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(edges, cv2.CV_64F, 0, 1, ksize=3)
    # Compute gradient magnitude and orientation
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    orientation = np.arctan2(grad_y, grad_x)
    # Flatten the magnitude and orientation into a feature vector
    eog_feature = np.concatenate([magnitude.flatten(), orientation.flatten()])

    return eog_feature

### LBP

In [59]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage.transform import resize

def LBP_extractor(image, target_size=(128, 128)):
    """
    Extract Local Binary Pattern (LBP) features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting LBP features.

    Returns:
        numpy.ndarray: Flattened LBP feature vector for the input image.
    """
    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)  # Adjust this based on your original image shape
    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    # Convert to grayscale
    gray_img = rgb2gray(resized_img)
    # Compute LBP features
    lbp = local_binary_pattern(gray_img, P=8, R=1, method='uniform')
    # Flatten the LBP feature vector
    return lbp.flatten()

### ORB

In [60]:
def ORB_extractor(image, target_size=(64, 64), n_features=500, fixed_length=2000):
    """
    Extract ORB features from a single image with fixed length.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting ORB features.
        n_features (int): Maximum number of features to retain.
        fixed_length (int): Fixed length for the ORB feature vector.

    Returns:
        numpy.ndarray: Flattened ORB descriptors with fixed length. If no descriptors are found, returns a zero vector.
    """
    orb = cv2.ORB_create(nfeatures=n_features)

    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)

    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    # Convert to grayscale
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(gray_img, None)
    # If no descriptors are found, return a zero vector
    if descriptors is None:
        descriptors = np.zeros((1, 32), dtype=np.uint8)
    # Flatten the descriptors
    descriptors = descriptors.flatten()

    # Ensure the feature vector has a fixed length
    if len(descriptors) < fixed_length:
        descriptors = np.pad(descriptors, (0, fixed_length - len(descriptors)), mode='constant')
    elif len(descriptors) > fixed_length:
        descriptors = descriptors[:fixed_length]

    return descriptors.flatten()

### HIST

In [61]:
def HIST_extractor(image, target_size=(64, 64), bins=(8, 8, 8)):
    """
    Extract color histogram (HIST) features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting HIST features.
        bins (tuple): Number of bins for each color channel (e.g., (8, 8, 8)).

    Returns:
        numpy.ndarray: Flattened HIST feature vector for the input image.
    """

    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)

    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    # Compute the color histogram for each channel (B, G, R)
    hist = cv2.calcHist([resized_img], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    # Normalize the histogram
    hist = cv2.normalize(hist, hist).flatten()

    return hist

### Combine the features

In [62]:
print_flag = False

def combined_feature_extractor(image):
    """
    Extract combined features (HOG, EOG, LBP, ORB, HIST, SIFT) from a single input image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).

    Returns:
        numpy.ndarray: Combined feature vector for the input image.
    """
    # # Extract features
    # static_features = image_statistics_extractor(image)  # Original image statistics
    # sift_features = SIFT_extractor(image)  # SIFT features
    # hog_features = HOG_extractor(image)   # HOG features
    # eog_features = EOG_extractor(image)   # EOG features
    # # lbp_features = LBP_extractor(image)   # LBP features
    # orb_features = ORB_extractor(image)   # ORB features
    # hist_features = HIST_extractor(image) # HIST features

    global print_flag

    # static_features = np.array(image_statistics_extractor(image), dtype=np.float32)
    sift_features = np.array(SIFT_extractor(image), dtype=np.float32)
    hog_features = np.array(HOG_extractor(image), dtype=np.float32)
    eog_features = np.array(EOG_extractor(image), dtype=np.float32)
    # lbp_features = np.array(LBP_extractor(image))
    orb_features = np.array(ORB_extractor(image), dtype=np.float32)
    hist_features = np.array(HIST_extractor(image), dtype=np.float32)

    # Combine features into a single vector
    combined_features = np.concatenate((
        # static_features,
        # sift_features,
        hog_features,
        # eog_features,
        # lbp_features,
        # orb_features,
        # hist_features,
    ))

    # Print feature shapes for debugging
    if print_flag == False:
        # print("Static features shape:", static_features.shape)
        print("HOG features shape:", hog_features.shape)
        print("EOG features shape:", eog_features.shape)
        print("ORB features shape:", orb_features.shape)
        print("HIST features shape:", hist_features.shape)
        print("SIFT features shape:", sift_features.shape)
        print_flag = True
    
    # del static_features
    del sift_features
    del hog_features
    del eog_features
    # del lbp_features
    del orb_features
    del hist_features
    
    # RECORD MANUALLY THE SHAPE OF EACH FEATURE
    # Static features shape: (12288,)
    # HOG features shape: (8100,)
    # EOG features shape: (8192,)
    # ORB features shape: (8000,)
    # HIST features shape: (512,)
    # SIFT features shape: (128,)

    return combined_features

### Convert training set with features

In [63]:
# Extract features from training and validation sets
X_train_features = [combined_feature_extractor(img) for img in tqdm(X_train, desc="Extracting features for X_train", total=len(X_train))]
X_val_features = [combined_feature_extractor(img) for img in tqdm(X_val, desc="Extracting features for X_val", total=len(X_val))]



# # Convert lists to numpy arrays
# X_train_featured = np.array(X_train_features)
# X_val_featured = np.array(X_val_features)

# Print shapes
# print("X_train_featured shape:", X_train_features.shape)
# print("X_val_featured shape:", X_val_features.shape)

Extracting features for X_train:   0%|          | 0/80000 [00:00<?, ?it/s]

HOG features shape: (12000,)
EOG features shape: (2048,)
ORB features shape: (2000,)
HIST features shape: (512,)
SIFT features shape: (128,)


Extracting features for X_val: 100%|██████████| 20000/20000 [01:08<00:00, 290.02it/s]


In [64]:

X_train_featured = np.array(X_train_features, dtype=np.float32)
X_val_featured = np.array(X_val_features, dtype=np.float32)
del X_train_features
del X_val_features

print("X_train_featured shape:", X_train_featured.shape)
print("X_val_featured shape:", X_val_featured.shape)



X_train_featured shape: (80000, 12000)
X_val_featured shape: (20000, 12000)


In [65]:
# for i in range(len(X_train_features)):
#     print(f"Image {i} has shape {X_train_features[i].shape}")
# for i in range(len(X_val_features)):
#     print(f"Image {i} has shape {X_val_features[i].shape}")

### SVM + PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_model = make_pipeline(
    StandardScaler(), 
    PCA(n_components=0.75, random_state=SEED), 
    SVC(C=8, kernel='rbf', gamma='scale',random_state=SEED))

svm_model.fit(X_train_featured, y_train)

y_pred_svm = svm_model.predict(X_val_featured)
accuracy_svm = accuracy_score(y_val, y_pred_svm)
print(f"SVM Validation Accuracy: {accuracy_svm:.4f}")

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
# import seaborn as sns
# # Confusion Matrix
# cm = confusion_matrix(y_val, y_pred_svm)
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_val), yticklabels=np.unique(y_val)) 
# plt.title("Confusion Matrix")
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_val, y_pred_svm))

MemoryError: Unable to allocate 7.15 GiB for an array with shape (80000, 12000) and data type float64

### Predict test data's class

In [ ]:
# from PIL import Image

# X_test_features = []
# X_test_resized = resize_images(X_test)
# for image in X_test_resized:
#     combined_features = combine_feature(image)
#     X_test_features.append(combined_features)
# X_test_features = np.array(X_test_features)
# print(f"Preprocessed test images shape: {X_test_features.shape}")

# # Predict labels
# y_test_pred = svm_clf.predict(X_test_features)

# # Save predictions to CSV
# test_df['label'] = y_test_pred
# submission_file = './submission.csv'
# test_df.to_csv(submission_file, index=False)

# print(f"Predictions saved to {submission_file}")